

# STEP1: 전처리

## 1-0: 전체 파일을 코인 별로 나누어 저장

<!-- 전체파일을 sort_values 해보자. (시간순으로 정렬) -->

In [1]:
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

# Change "current directory"
new_dir = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
# new_dir = 'C:\\Users\\user\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(new_dir)

import numpy as np
import csv
import dask.dataframe as dd # This is a main package to process a large csv file.
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
import pandas as pd 


In [2]:
# data_id = 202302230905 
# data_id = 202311281936 
# data_id = 202303271051
# data_id = 202312021632 
# data_id = 202312081015
# data_id = '202312122122'
# 202302230905
#  202303271051 # 실질적으로는 12/16부터 2/26까지 데이터 존재.

# in_csv ="D:\\ticker_data_{}.csv".format(data_id)
in_csv ="D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\Data\\ticker_data_{}.csv".format(data_id)

dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# Read the CSV file using a Dask DataFrame
df = dd.read_csv(in_csv, dtype=dtype) 


NameError: name 'data_id' is not defined

In [3]:
## 수집된 파일에는 n개의 코인이 함께 저장되어 있음. 따라서, Separate files by COIN

coin_list = ['BTC'] #, 'ETH', 'DOGE', 'XRP', 'LINK', 'AVAX', 'SOL']

for coin in coin_list:
    globals()["df_temp"] = globals()["df"][globals()["df"].code == f'KRW-{coin}']
    # with open("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv".format(coin, data_id), 'wb') as file:
    #     pickle.dump(df_temp, file)
    df_temp.to_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv".format(coin, data_id))
    print('Complete: {}'.format(coin))


KeyError: 'df'

## 1-1: 코인별로 나누어진 파일을, 시간순으로 sorting 후 다시 저장
### duplicates 제거 및 30초/60초 ticker 값을 제거


In [ ]:
import os
#USE ONLY ONE OF THESE:
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

# Change "current directory"
new_dir = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(new_dir)

# Import packages
import numpy as np
import csv
import dask.dataframe as dd # This is a main package to process a large csv file.
import pyarrow as pa
import pyarrow.parquet as pq
# import pandas as pd 

In [ ]:
dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# data_id = 202303271051 # 해당 틱데이터 커버 기간: 22년 12월 16일 오후 9시 ~ 23년 2월 26일 오전 4시
# coin_list = [ 'BTC',  'ETH',  'DOGE', 'XRP'] 

for coin in coin_list:

    # Read dask "part" files
    part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv\\".format(coin, data_id)
    df = dd.read_csv( part_files_path + '*.part', dtype=dtype)    
    # df = dd.read_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv".format(coin, data_id))
    # with open('.\\output\\t_data_{}_{}.pkl'.format(coin, data_id), 'rb') as file:
    #     loaded_df = pickle.load(file)

    # Sort dataframe
    df = df.sort_values('sys_datetime', ascending = True, na_position = 'last')
    print('Complete: Sorting')
    
    # Write the sorted DataFrame to a new dask-type csv file
    df.to_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_data_{}_{}.csv".format(coin, data_id), index=False)

    # with open("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_data_{}_{}.csv".format(coin, data_id), 'wb') as file:
    #     pickle.dump(loaded_df, file)

    # # Write the sorted DataFrame to a new parquet file
    # df.to_parquet('s_t_data_{}_{}.parquet'.format(coin,data_id), engine='pyarrow')

## Note: 용량 부족 메시지가 떠서, 한 코인씩 돌림. (for 문을 돌리지 못함.)    
## Note: parquet으로는 저장되지 않음 (메모리 문제라는 메시지가 뜸.)

Complete: Sorting


### duplicates 제거 및 30초/60초 ticker 값을 제거 (단위 시간마다 신규 거래가 없더라도 ticker 값이 들어온다)

In [18]:
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
new_dir = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(new_dir)
import numpy as np
import csv
import dask.dataframe as dd
import pyarrow as pa
import pyarrow.parquet as pq
import gc


dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

data_id = '202312122122'
# data_id = '202303271051' # 해당 틱데이터 커버 기간: 22년 12월 16일 오후 9시 ~ 23년 2월 26일 오전 4시
# coin_list = ['BTC', 'ETH',  'DOGE', 'XRP'] 
coin_list = ['BTC'] 
for coin in coin_list:

    # with open('.\\output\\s_t_data_{}_{}.csv'.format(coin, data_id), 'rb') as file:
    #     loaded_df = pickle.load(file)

    ### Read dask "part" failes
    part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_data_{}_{}.csv\\".format(coin, data_id)
    df = dd.read_csv( part_files_path + '*.part', dtype=dtype)    
    df = df.compute()

    gc.collect()

    ### Drop duplicates
    print("df.shape 1 :", df.shape)
    df.drop_duplicates(subset=[ 'type_websocket', 'timestamp','sys_datetime'], keep='first', inplace=True, ignore_index=True)
    print("df.shape 2 :", df.shape)
    df.drop_duplicates(subset=[ 'type_websocket',  'opening_price',  'high_price',   'low_price',  'trade_price',  'prev_closing_price', 'change',  'change_price',  'signed_change_price',  'change_rate',  'signed_change_rate',  'trade_volume',  'acc_trade_volume',   'acc_trade_price',    'trade_date', 'ask_bid',  'acc_ask_volume',  'acc_bid_volume',  'highest_52_week_price', 'highest_52_week_date',  'lowest_52_week_price', 'lowest_52_week_date', 'market_state',  'is_trading_suspended'   , 'total_ask_size',  'total_bid_size',  'orderbook_ap_0',  'orderbook_as_0',  'orderbook_bp_0',  'orderbook_bs_0',  'orderbook_ap_1',  'orderbook_as_1',  'orderbook_bp_1',  'orderbook_bs_1',  'orderbook_ap_2',  'orderbook_as_2',  'orderbook_bp_2',  'orderbook_bs_2',  'orderbook_ap_3',  'orderbook_as_3',  'orderbook_bp_3',  'orderbook_bs_3',  'orderbook_ap_4',  'orderbook_as_4',  'orderbook_bp_4',  'orderbook_bs_4',  'orderbook_ap_5',  'orderbook_as_5',  'orderbook_bp_5',  'orderbook_bs_5',  'orderbook_ap_6',  'orderbook_as_6',  'orderbook_bp_6',  'orderbook_bs_6',  'orderbook_ap_7',  'orderbook_as_7',  'orderbook_bp_7',  'orderbook_bs_7',  'orderbook_ap_8',  'orderbook_as_8',  'orderbook_bp_8',  'orderbook_bs_8',  'orderbook_ap_9',  'orderbook_as_9',  'orderbook_bp_9',  'orderbook_bs_9',  'orderbook_ap_10',  'orderbook_as_10',  'orderbook_bp_10',  'orderbook_bs_10',  'orderbook_ap_11',  'orderbook_as_11',  'orderbook_bp_11',  'orderbook_bs_11',  'orderbook_ap_12',  'orderbook_as_12',  'orderbook_bp_12',  'orderbook_bs_12',  'orderbook_ap_13',  'orderbook_as_13',  'orderbook_bp_13',  'orderbook_bs_13',  'orderbook_ap_14',  'orderbook_as_14',  'orderbook_bp_14',  'orderbook_bs_14'    ], keep='first', inplace=True, ignore_index=True)
    print("df.shape 3 :", df.shape)

    print('Complete: Drop duplicates')

    ### Write the sorted DataFrame to a new dask-type csv file
    df.to_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_d_data_{}_{}.csv".format(coin, data_id), index=False)
    print('Complete: File Saving')


df.shape 1 : (10606368, 99)
df.shape 2 : (10606368, 99)
df.shape 3 : (9506748, 99)
Complete: Drop duplicates
Complete: File Saving


## 1-2(b): 각 코인의 틱데이터를 10분씩 묶는 전처리 작업 (rolling window)

### Setting & Running => Output: {}_sum_both_10m.csv

In [1]:
# Version Number

data_id = 202303271051
# data_id = 202312122122
coin = 'BTC'

### Libraries and key functions
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(working_directory)

import numpy as np
import csv
import gc
import dask.dataframe as dd
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from contextlib import contextmanager  # 컨텍스트 관리자를 사용하기 위한 contextlib 모듈을 가져옵니다.
import time  # 시간 관련 기능을 사용하기 위한 time 모듈을 가져옵니다.


def log_return(series: np.ndarray):
    return np.log(series).diff()

def realized_volatility(series):
    return np.sqrt(np.sum(series ** 2))

def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('.'.join([prefix] + list(c)))
    return ret

@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f'[{name}] {elapsed: .3f}초')


In [26]:
import os
#USE ONLY ONE OF THESE:
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
# Change "current directory"
working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(working_directory)

import numpy as np
import csv
import dask.dataframe as dd # This is a main package to process a large csv file.
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd 
import gc

dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# with open('.\\output\\s_t_d_data_{}_{}.pkl'.format(coin, data_id), 'rb') as file:
#        df = pickle.load(file)
# s_t_d_data_BTC_202312122122

## 저장이 Dask dataframe으로 되어 있으면 아래 코드를 실행
part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_d_data_{}_{}.csv\\".format(coin, data_id)
df = dd.read_csv( part_files_path + '*.part', dtype=dtype) 

# # ## 저장이 pandas dataframe으로 되어 있으면 아래 코드를 실행
# part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_d_data_{}_{}.csv".format(coin, data_id)
# df = pd.read_csv( part_files_path , dtype=dtype) 


In [27]:
## df를 ticker와 orderbook으로 구분.

import gc

vn = 8 ### 같은 데이터셋 안에서도 vn이 다를 수 있음.. (feature의 변화라던지..) 

df = df.compute() ############################################# dask dataframe 이면 pandas dataframe으로 바꾸어주어야.

# Convert float64 columns to float32
float_columns = df.select_dtypes(include=['float64'])
df = df.assign(**{col: df[col].astype('float32') for col in float_columns.columns})
int64_columns = df.select_dtypes(include=['int64'])
df = df.assign(**{col: df[col].astype('int32') for col in int64_columns.columns})

df['datetime'] = df['sys_datetime'] # datetime에는 문제가 있는 것 같다.

df_orderbook = df[df['type_websocket']=="orderbook"]
df_ticker = df[df['type_websocket']=="ticker"]

df_ticker = df_ticker.set_index('sys_datetime', drop=False)
df_orderbook = df_orderbook.set_index('sys_datetime', drop=False)

gc.collect()

0

In [28]:
df_orderbook.tail(3)

,Unnamed: 0,type_websocket,datetime,code,opening_price,high_price,low_price,trade_price,prev_closing_price,change,change_price,signed_change_price,change_rate,signed_change_rate,trade_volume,acc_trade_volume,acc_trade_volume_24h,acc_trade_price,acc_trade_price_24h,trade_date,trade_time,trade_timestamp,ask_bid,acc_ask_volume,acc_bid_volume,highest_52_week_price,highest_52_week_date,lowest_52_week_price,lowest_52_week_date,market_state,is_trading_suspended,delisting_date,market_warning,timestamp,stream_type,sys_datetime,sequential_id,total_ask_size,total_bid_size,orderbook_ap_0,orderbook_as_0,orderbook_bp_0,orderbook_bs_0,orderbook_ap_1,orderbook_as_1,orderbook_bp_1,orderbook_bs_1,orderbook_ap_2,orderbook_as_2,orderbook_bp_2,orderbook_bs_2,orderbook_ap_3,orderbook_as_3,orderbook_bp_3,orderbook_bs_3,orderbook_ap_4,orderbook_as_4,orderbook_bp_4,orderbook_bs_4,orderbook_ap_5,orderbook_as_5,orderbook_bp_5,orderbook_bs_5,orderbook_ap_6,orderbook_as_6,orderbook_bp_6,orderbook_bs_6,orderbook_ap_7,orderbook_as_7,orderbook_bp_7,orderbook_bs_7,orderbook_ap_8,orderbook_as_8,orderbook_bp_8,orderbook_bs_8,orderbook_ap_9,orderbook_as_9,orderbook_bp_9,orderbook_bs_9,orderbook_ap_10,orderbook_as_10,orderbook_bp_10,orderbook_bs_10,orderbook_ap_11,orderbook_as_11,orderbook_bp_11,orderbook_bs_11,orderbook_ap_12,orderbook_as_12,orderbook_bp_12,orderbook_bs_12,orderbook_ap_13,orderbook_as_13,orderbook_bp_13,orderbook_bs_13,orderbook_ap_14,orderbook_as_14,orderbook_bp_14,orderbook_bs_14
sys_datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-26 04:24:25.112,13614,orderbook,2023-02-26 04:24:25.112,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1978880974,NaN,2023-02-26 04:24:25.112,NaN,5.512517,6.228781,30599000.0,0.032173,30565000.0,0.283228,30600000.0,1.049995,30564000.0,0.509400,30601000.0,0.049296,30563000.0,0.743168,30605000.0,0.496,30562000.0,0.653600,30606000.0,0.381237,30560000.0,0.000164,30607000.0,0.000622,30553000.0,0.215000,30608000.0,0.000653,30551000.0,0.934944,30610000.0,1.95998,30550000.0,0.000164,30611000.0,1.030641,30548000.0,0.000344,30613000.0,0.000566,30546000.0,1.535377,30617000.0,0.039802,30545000.0,0.185,30618000.0,0.006859,30544000.0,0.85900,30619000.0,0.001175,30542000.0,0.006680,30620000.0,0.172306,30541000.0,0.026087,30621000.0,0.291212,30540000.0,0.276624
2023-02-26 04:24:25.262,13616,orderbook,2023-02-26 04:24:25.262,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1978880794,NaN,2023-02-26 04:24:25.262,NaN,5.512517,5.501081,30599000.0,0.032173,30565000.0,0.283228,30600000.0,1.049995,30564000.0,0.509400,30601000.0,0.049296,30563000.0,0.015468,30605000.0,0.496,30562000.0,0.653600,30606000.0,0.381237,30560000.0,0.000164,30607000.0,0.000622,30553000.0,0.215000,30608000.0,0.000653,30551000.0,0.934944,30610000.0,1.95998,30550000.0,0.000164,30611000.0,1.030641,30548000.0,0.000344,30613000.0,0.000566,30546000.0,1.535377,30617000.0,0.039802,30545000.0,0.185,30618000.0,0.006859,30544000.0,0.85900,30619000.0,0.001175,30542000.0,0.006680,30620000.0,0.172306,30541000.0,0.026087,30621000.0,0.291212,30540000.0,0.276624
2023-02-26 04:24:25.462,28105,orderbook,2023-02-26 04:24:25.462,KRW-BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1978880614,NaN,2023-02-26 04:24:25.462,NaN,5.512517,5.697045,30599000.0,0.032173,30565000.0,0.283228,30600000.0,1.049995,30563000.0,0.524868,30601000.0,0.049296,30562000.0,0.653600,30605000.0,0.496,30560000.0,0.000164,30606000.0,0.381237,30553000.0,0.215000,30607000.0,0.000622,30551000.0,0.934944,30608000.0,0.000653,30550000.0,0.000164,30610000.0,1.95998,30548000.0,0.000344,30611000.0,1.030641,30546000.0,1.535377,30613000.0,0.000566,30545000.0,0.185000,30617000.0,0.039802,30544000.0,0.859,30618000.0,0.006859,30542000.0,0.

In [29]:
df_ticker.tail(3)

,Unnamed: 0,type_websocket,datetime,code,opening_price,high_price,low_price,trade_price,prev_closing_price,change,change_price,signed_change_price,change_rate,signed_change_rate,trade_volume,acc_trade_volume,acc_trade_volume_24h,acc_trade_price,acc_trade_price_24h,trade_date,trade_time,trade_timestamp,ask_bid,acc_ask_volume,acc_bid_volume,highest_52_week_price,highest_52_week_date,lowest_52_week_price,lowest_52_week_date,market_state,is_trading_suspended,delisting_date,market_warning,timestamp,stream_type,sys_datetime,sequential_id,total_ask_size,total_bid_size,orderbook_ap_0,orderbook_as_0,orderbook_bp_0,orderbook_bs_0,orderbook_ap_1,orderbook_as_1,orderbook_bp_1,orderbook_bs_1,orderbook_ap_2,orderbook_as_2,orderbook_bp_2,orderbook_bs_2,orderbook_ap_3,orderbook_as_3,orderbook_bp_3,orderbook_bs_3,orderbook_ap_4,orderbook_as_4,orderbook_bp_4,orderbook_bs_4,orderbook_ap_5,orderbook_as_5,orderbook_bp_5,orderbook_bs_5,orderbook_ap_6,orderbook_as_6,orderbook_bp_6,orderbook_bs_6,orderbook_ap_7,orderbook_as_7,orderbook_bp_7,orderbook_bs_7,orderbook_ap_8,orderbook_as_8,orderbook_bp_8,orderbook_bs_8,orderbook_ap_9,orderbook_as_9,orderbook_bp_9,orderbook_bs_9,orderbook_ap_10,orderbook_as_10,orderbook_bp_10,orderbook_bs_10,orderbook_ap_11,orderbook_as_11,orderbook_bp_11,orderbook_bs_11,orderbook_ap_12,orderbook_as_12,orderbook_bp_12,orderbook_bs_12,orderbook_ap_13,orderbook_as_13,orderbook_bp_13,orderbook_bs_13,orderbook_ap_14,orderbook_as_14,orderbook_bp_14,orderbook_bs_14
sys_datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-26 04:24:22.922,13555,ticker,2023-02-26 04:24:22.922,KRW-BTC,30868000.0,30908000.0,30291000.0,30599000.0,30877000.0,FALL,278000.0,-278000.0,0.009003,-0.009003,0.003940,3955.480225,5009.032715,1.210514e+11,1.535216e+11,20230224.0,192849,1.677353e+12,BID,2128.713135,1826.766968,57678000.0,2022-03-28,20700000.0,2022-12-30,ACTIVE,0.0,NaN,NONE,-1978883128,REALTIME,2023-02-26 04:24:22.922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-26 04:24:22.925,28043,ticker,2023-02-26 04:24:22.925,KRW-BTC,30868000.0,30908000.0,30291000.0,30599000.0,30877000.0,FALL,278000.0,-278000.0,0.009003,-0.009003,0.000440,3955.480713,5009.032715,1.210514e+11,1.535216e+11,20230224.0,192849,1.677353e+12,BID,2128.713135,1826.767456,57678000.0,2022-03-28,20700000.0,2022-12-30,ACTIVE,0.0,NaN,NONE,-1978883126,REALTIME,2023-02-26 04:24:22.925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-26 04:24:24.205,28079,ticker,2023-02-26 04:24:24.205,KRW-BTC,30868000.0,30908000.0,30291000.0,30599000.0,30877000.0,FALL,278000.0,-278000.0,0.009003,-0.009003,0.016447,3955.497070,5009.032715,1.210519e+11,1.535216e+11,20230224.0,192850,1.677353e+12,BID,2128.713135,1826.783813,57678000.0,2022-03-28,20700000.0,2022-12-30,ACTIVE,0.0,NaN,NONE,-1978881852,REALTIME,2023-02-26 04:24:24.205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Feature Engineering (TICKER 부문)

In [47]:
df_ticker_pd = df_ticker.copy()
df_ticker_pd['datetime'] = pd.to_datetime(df_ticker_pd['datetime'])
df_ticker_pd.set_index('datetime', inplace=False)
df_ticker_pd['dt_30s'] = df_ticker_pd['datetime'].dt.floor('30s').dt.strftime('%Y-%m-%d %H:%M:%S')


In [56]:
df_ticker_pd.index[0]

'2022-12-16 21:05:37.281'

In [40]:
temp_index = df_ticker_pd['dt_30s'].unique().tolist()
# Calculate the expected 30-second intervals & Check for missing intervals
expected_index = pd.date_range(start=temp_index[0], end=temp_index[-1], freq='30S')
missing_intervals = expected_index.difference(temp_index)
if missing_intervals.empty:
    print("No missing 30-second intervals.")
else:
    print("Missing 30-second intervals:")
    print(missing_intervals)
    print('missing_intervals type:', type(missing_intervals))
    print('missing_intervals length:', len(missing_intervals))

print(len(expected_index))
print(expected_index[:10])

205358
DatetimeIndex(['2022-12-16 21:05:30', '2022-12-16 21:06:00',
               '2022-12-16 21:06:30', '2022-12-16 21:07:00',
               '2022-12-16 21:07:30', '2022-12-16 21:08:00',
               '2022-12-16 21:08:30', '2022-12-16 21:09:00',
               '2022-12-16 21:09:30', '2022-12-16 21:10:00'],
              dtype='datetime64[ns]', freq='30S')


In [82]:
def feature_engineering_ticker(df_ticker):

    df_ticker_pd = df_ticker.copy()
    df_ticker_pd.reset_index(inplace=True, drop=True)
    df_ticker_pd['datetime'] = pd.to_datetime(df_ticker_pd['datetime'])
    df_ticker_pd.set_index('datetime', inplace=True, drop=False)

    # 30 seconds summarization
    df_ticker_pd['dt_30s'] = df_ticker_pd['datetime'].dt.floor('30s').dt.strftime('%Y-%m-%d %H:%M:%S')

    # 30 seconds for features. 10 minutes for target var.
    window_size = pd.Timedelta(minutes=10, seconds=30)

    # Create an empty list to store features for each rolling window
    realized_vol_next10m = [] #  target 1
    # num_trades = [] # tick-level data is not complete. So, its meaning is little.
    num_rows = []
    lowest_return = []
    trade_vol = []
    volume_power = []
    highest_return = []
    high_low_gap = []
    ending_price = []
    returns = []
    returns_next10m = [] #  target 2
    # trade_vol_top1p_ratio = []
    # volume_power_top1p = []
    highest_price = []
    lowest_price = []
    beginning_price = []

    temp_index = df_ticker_pd['dt_30s'].unique().tolist()
    # Calculate the expected 30-second intervals & Check for missing intervals
    expected_index = pd.date_range(start=temp_index[0], end=temp_index[-1], freq='30S')
    missing_intervals = expected_index.difference(temp_index)
    if missing_intervals.empty:
        print("No missing 30-second intervals.")
    else:
        print("Missing 30-second intervals:")
        print(missing_intervals)
        print('missing_intervals type:', type(missing_intervals))
        print('missing_intervals length:', len(missing_intervals))

    # Define the time range
    start_time = pd.to_datetime('00:00:00').time()
    end_time = pd.to_datetime('06:00:00').time()

    window_start2 = []
    window_end2 = []

    # Iterate through each time window using the rolling method
    for window_start in temp_index[:-21]:
        window_start = pd.Timestamp(window_start)
        window_end = window_start + window_size

        if  (start_time <= window_start.time() <= end_time) or  (start_time <= window_end.time() <= end_time):
            continue 
        
        # print('type(df_ticker_pd.index)', type(df_ticker_pd.index))
        # print('type(window_start):', type(window_start))

        rolling_window_data = df_ticker_pd.loc[(df_ticker_pd.index >= window_start) & (df_ticker_pd.index < window_end)]

        # print('type(window_start.index)', type(window_start))
        # print('type(window_start+pd.Timedelta(seconds=30)):', type(window_start+pd.Timedelta(seconds=30)))

        rolling_window_data_target = rolling_window_data.loc[(rolling_window_data.index >= window_start+pd.Timedelta(seconds=30)) & (rolling_window_data.index < window_start+pd.Timedelta(seconds=30) + pd.Timedelta(minutes=10))]
        rolling_window_data_feature = rolling_window_data.loc[(rolling_window_data.index >= window_start) & (rolling_window_data.index < window_start+pd.Timedelta(seconds=30))]

        try: 
            previous_price_target = rolling_window_data_target['trade_price'].iloc[0]
            previous_price_feature = rolling_window_data_feature['trade_price'].iloc[0]

        except Exception as e:
            print("No previous price.")
            continue

        window_start2.append(window_start)
        window_end2.append(window_start+pd.Timedelta(seconds=30))
        
        rolling_window_data_target.loc[:, 'log_return_from_pp'] = np.log(rolling_window_data_target['trade_price'] / previous_price_target)
        rolling_window_data_feature.loc[:, 'log_return_from_pp'] = np.log(rolling_window_data_feature['trade_price'] / previous_price_feature)

        returns2 = rolling_window_data_target['log_return_from_pp']
        squared_returns = np.square(returns2)
        average_squared_returns = np.mean(squared_returns)
        realized_vol_next10m_temp = np.sqrt(average_squared_returns)
        realized_vol_next10m.append(realized_vol_next10m_temp)

        returns_next10m_temp = np.log(rolling_window_data_target['trade_price'].iloc[-1] / previous_price_target)
        returns_next10m.append(returns_next10m_temp)

        num_rows_temp = rolling_window_data_feature.shape[0]
        num_rows.append(num_rows_temp)

        lowest_return_temp = rolling_window_data_feature['log_return_from_pp'].min()
        lowest_return.append(lowest_return_temp)

        highest_return_temp = rolling_window_data_feature['log_return_from_pp'].max()
        highest_return.append(highest_return_temp) 

        lowest_price_temp = rolling_window_data_feature['trade_price'].min()
        lowest_price.append(lowest_price_temp)
        highest_price_temp = rolling_window_data_feature['trade_price'].max()
        highest_price.append(highest_price_temp)

        high_low_gap_temp = highest_return_temp - lowest_return_temp
        high_low_gap.append(high_low_gap_temp)

        trade_vol_temp = rolling_window_data_feature['trade_volume'].sum()
        trade_vol.append(trade_vol_temp)

        ask_sum = rolling_window_data_feature[rolling_window_data['ask_bid'] == 'ASK']['trade_volume'].sum()
        bid_sum = rolling_window_data_feature[rolling_window_data['ask_bid'] == 'BID']['trade_volume'].sum()
        volume_power_temp = (bid_sum - ask_sum) / trade_vol_temp  # 후에 전체 거래량으로 나누어줄 예정이다.
        volume_power.append(volume_power_temp)

        beginning_price_temp = rolling_window_data_feature['trade_price'].iloc[0]
        beginning_price.append(beginning_price_temp)

        ending_price_temp = rolling_window_data_feature['trade_price'].iloc[-1]
        ending_price.append(ending_price_temp)

        returns_temp = rolling_window_data_feature['trade_price'].iloc[-1]/previous_price_feature
        returns.append(returns_temp)

    result_df = pd.DataFrame({
        'window_start': window_start2,
        'window_end': window_end2,
        'num_rows': num_rows,
        'lowest_return': lowest_return,
        'highest_return': highest_return,
        'high_low_gap': high_low_gap,
        'trade_vol': trade_vol,
        'volume_power': volume_power,
        'beginning_price': beginning_price,
        'ending_price': ending_price,
        'lowest_price': lowest_price,
        'highest_price': highest_price,
        'returns': returns,
        'returns_next10m': returns_next10m,
        'realized_vol_next10m': realized_vol_next10m,
    })

    # time_id (for later usage)
    result_df['time_id'] = result_df['window_start'] # .dt.floor('30s').dt.strftime('%Y-%m-%d %H:%M:%S')

    return result_df


### Feature Engineering (ORDERBOOK 부문)

In [86]:
def feature_engineering_orderbook(df_orderbook):

    df_orderbook_pd = df_orderbook_pd.copy()
    df_orderbook_pd.reset_index(inplace=True, drop=True)
    df_orderbook_pd['datetime'] = pd.to_datetime(df_orderbook_pd['datetime'])
    df_orderbook_pd.set_index('datetime', inplace=True, drop=False)

    # 30 seconds summarization
    df_orderbook_pd['dt_30s'] = df_orderbook_pd['datetime'].dt.floor('30s').dt.strftime('%Y-%m-%d %H:%M:%S')

    # Create an empty list to store features for each rolling window

    window_start2 = []
    window_end2 = []
    
    # store orderbook information at the end of the 30s window.
    ob_end_ap = {}
    ob_end_as = {}
    ob_end_bp = {}
    ob_end_bs = {}

    for i in range(15):
        ob_end_ap[i] = []
        ob_end_as[i] = []
        ob_end_bp[i] = []
        ob_end_bs[i] = []

    ob_end_bias_0 = []
    ob_end_bias_1 = []
    ob_end_bias_4 = []
    ob_end_bidask_spread = []
    ob_end_liq_0 = []
    ob_end_liq_1 = []
    ob_end_liq_4 = []
    highest_possible_return = [] 

    # 30 seconds for features. 10 minutes for target var.
    window_size = pd.Timedelta(minutes=10, seconds=30)

    temp_index = df_orderbook_pd['dt_30s'].unique().tolist()

    # Calculate the expected 30-second intervals & Check for missing intervals
    expected_index = pd.date_range(start=temp_index[0], end=temp_index[-1], freq='30S')
    missing_intervals = expected_index.difference(temp_index)
    if missing_intervals.empty:
        print("No missing 30-second intervals.")
    else:
        print("Missing 30-second intervals:")
        print(missing_intervals)
        print('missing_intervals type:', type(missing_intervals))
        print('missing_intervals length:', len(missing_intervals))

    # Define the time range (ingorning early morning data)
    start_time = pd.to_datetime('00:00:00').time()
    end_time = pd.to_datetime('06:00:00').time()

    window_start2 = []
    window_end2 = []

    for window_start in temp_index[:-21]:
        
        window_start = pd.Timestamp(window_start)
        window_end = window_start + window_size

        if  (start_time <= window_start.time() <= end_time) or  (start_time <= window_end.time() <= end_time):
            continue 

        rolling_window_data = df_orderbook_pd.loc[(df_orderbook_pd.index >= window_start) & (df_orderbook_pd.index < window_end)]
        rolling_window_data_target = rolling_window_data.loc[(rolling_window_data.index >= window_start+pd.Timedelta(seconds=30)) & (rolling_window_data.index < window_start+pd.Timedelta(seconds=30) + pd.Timedelta(minutes=10))]
        rolling_window_data_feature = rolling_window_data.loc[(rolling_window_data.index >= window_start) & (rolling_window_data.index < window_start+pd.Timedelta(seconds=30))]

        window_start2.append(window_start)
        window_end2.append(window_start+pd.Timedelta(seconds=30))
        
        # Check the progress
        if j % 1000 ==0:
            print("main iteration:", j)

        last_order_book_feature = rolling_window_data_feature.iloc[-1]
        first_order_book_target = rolling_window_data_target.iloc[0]
        last_order_book_target = rolling_window_data_target.iloc[-1]

        best_buying_price_at_start = first_order_book_target['orderbook_ap_0']
        best_selling_price = rolling_window_data_target['orderbook_bp_0'].max()
        highest_possible_return_temp = best_selling_price / best_buying_price_at_start
        highest_possible_return.append(highest_possible_return_temp) 

        for i in range(15):
            ob_end_ap[i].append(last_order_book_feature[f'orderbook_ap_{i}'])
            ob_end_as[i].append(last_order_book_feature[f'orderbook_as_{i}'])
            ob_end_bp[i].append(last_order_book_feature[f'orderbook_bp_{i}'])
            ob_end_bs[i].append(last_order_book_feature[f'orderbook_bs_{i}'])

        ob_end_bias_0_temp  = last_order_book_feature['orderbook_as_0'].sum() / last_order_book_feature['orderbook_bs_0'].sum() 
        ob_end_bias_1_temp  = (last_order_book_feature['orderbook_as_0']+last_order_book_feature['orderbook_as_1']).sum() / (last_order_book_feature['orderbook_bs_0']+last_order_book_feature['orderbook_bs_1']).sum() 
        ob_end_bias_4_temp  = (last_order_book_feature['orderbook_as_0']+last_order_book_feature['orderbook_as_1']+last_order_book_feature['orderbook_as_2']+last_order_book_feature['orderbook_as_3']+last_order_book_feature['orderbook_as_4']).sum()/ (last_order_book_feature['orderbook_bs_0']+last_order_book_feature['orderbook_bs_1']+last_order_book_feature['orderbook_bs_2']+last_order_book_feature['orderbook_bs_3']+last_order_book_feature['orderbook_bs_4']).sum()
        ob_end_bias_0.append(ob_end_bias_0_temp)
        ob_end_bias_1.append(ob_end_bias_1_temp)
        ob_end_bias_4.append(ob_end_bias_4_temp)

        ob_price_diff = []
        for i in range(14):
            ob_price_diff.append(last_order_book_feature[f'orderbook_bp_{i}'] - last_order_book_feature[f'orderbook_bp_{i+1}'])
            ob_price_diff.append(last_order_book_feature[f'orderbook_ap_{i+1}'] - last_order_book_feature[f'orderbook_ap_{i}'])
        tick_size = min(ob_price_diff)
        ob_end_bidask_spread_temp = (last_order_book_feature['orderbook_ap_0'] - last_order_book_feature['orderbook_bp_0'])/tick_size
        ob_end_bidask_spread.append(ob_end_bidask_spread_temp)

        wap_1 = (last_order_book_feature['orderbook_bp_0'] * last_order_book_feature['orderbook_as_0'] + last_order_book_feature['orderbook_ap_0'] * last_order_book_feature['orderbook_bs_0'])/(last_order_book_feature['orderbook_bs_0'] + last_order_book_feature['orderbook_as_0'])
        data_liq = {}
        temp = 0
        for i in range(5):
            temp += last_order_book_feature[f'orderbook_bs_{i}']/(wap_1 - last_order_book_feature[f'orderbook_bp_{i}']) + last_order_book_feature[f'orderbook_as_{i}']/(last_order_book_feature[f'orderbook_ap_{i}'] - wap_1)
            if i+1 in [1, 2, 5]: # , 10, 15
                variable_name = f'liq_{i}'
                data_liq[variable_name] = temp
        ob_end_liq_0.append(data_liq['liq_0'])
        ob_end_liq_1.append(data_liq['liq_1'])
        ob_end_liq_4.append(data_liq['liq_4'])

        j= j+1
        gc.collect()

    # Create a new DataFrame to store the results
    result_df_ob = pd.DataFrame({
        'window_start': window_start2,
        'window_end': window_end2,
        'ob_end_ap_0': ob_end_ap[0],
        'ob_end_as_0': ob_end_as[0],
        'ob_end_bp_0': ob_end_bp[0],
        'ob_end_bs_0': ob_end_bs[0],
        'ob_end_ap_1': ob_end_ap[1],
        'ob_end_as_1': ob_end_as[1],
        'ob_end_bp_1': ob_end_bp[1],
        'ob_end_bs_1': ob_end_bs[1],
        'ob_end_ap_2': ob_end_ap[2],
        'ob_end_as_2': ob_end_as[2],
        'ob_end_bp_2': ob_end_bp[2],
        'ob_end_bs_2': ob_end_bs[2],
        'ob_end_ap_3': ob_end_ap[3],
        'ob_end_as_3': ob_end_as[3],
        'ob_end_bp_3': ob_end_bp[3],
        'ob_end_bs_3': ob_end_bs[3],
        'ob_end_ap_4': ob_end_ap[4],
        'ob_end_as_4': ob_end_as[4],
        'ob_end_bp_4': ob_end_bp[4],
        'ob_end_bs_4': ob_end_bs[4],
        'ob_end_ap_5': ob_end_ap[5],
        'ob_end_as_5': ob_end_as[5],
        'ob_end_bp_5': ob_end_bp[5],
        'ob_end_bs_5': ob_end_bs[5],
        'ob_end_ap_6': ob_end_ap[6],
        'ob_end_as_6': ob_end_as[6],
        'ob_end_bp_6': ob_end_bp[6],
        'ob_end_bs_6': ob_end_bs[6],
        'ob_end_ap_7': ob_end_ap[7],
        'ob_end_as_7': ob_end_as[7],
        'ob_end_bp_7': ob_end_bp[7],
        'ob_end_bs_7': ob_end_bs[7],
        'ob_end_ap_8': ob_end_ap[8],
        'ob_end_as_8': ob_end_as[8],
        'ob_end_bp_8': ob_end_bp[8],
        'ob_end_bs_8': ob_end_bs[8],
        'ob_end_ap_9': ob_end_ap[9],
        'ob_end_as_9': ob_end_as[9],
        'ob_end_bp_9': ob_end_bp[9],
        'ob_end_bs_9': ob_end_bs[9],
        'ob_end_ap_10': ob_end_ap[10],
        'ob_end_as_10': ob_end_as[10],
        'ob_end_bp_10': ob_end_bp[10],
        'ob_end_bs_10': ob_end_bs[10],
        'ob_end_ap_11': ob_end_ap[11],
        'ob_end_as_11': ob_end_as[11],
        'ob_end_bp_11': ob_end_bp[11],
        'ob_end_bs_11': ob_end_bs[11],
        'ob_end_ap_12': ob_end_ap[12],
        'ob_end_as_12': ob_end_as[12],
        'ob_end_bp_12': ob_end_bp[12],
        'ob_end_bs_12': ob_end_bs[12],
        'ob_end_ap_13': ob_end_ap[13],
        'ob_end_as_13': ob_end_as[13],
        'ob_end_bp_13': ob_end_bp[13],
        'ob_end_bs_13': ob_end_bs[13],
        'ob_end_ap_14': ob_end_ap[14],
        'ob_end_as_14': ob_end_as[14],
        'ob_end_bp_14': ob_end_bp[14],
        'ob_end_bs_14': ob_end_bs[14],
        'ob_end_bias_0': ob_end_bias_0,
        'ob_end_bias_1': ob_end_bias_1,
        'ob_end_bias_4': ob_end_bias_4,
        'ob_end_bidask_spread': ob_end_bidask_spread,
        'ob_end_liq_0': ob_end_liq_0,
        'ob_end_liq_1': ob_end_liq_1,
        'ob_end_liq_4': ob_end_liq_4,
        'highest_possible_return': highest_possible_return,
    })

    # time_id (for later usage)
    result_df_ob['time_id'] = result_df_ob['window_start'] # .dt.floor('30s').dt.strftime('%Y-%m-%d %H:%M:%S')

    return result_df_ob


In [5]:
# df_features_ticker = pd.read_csv(working_directory+"output\\BTC_sum_ticker_10m_202303271051_v7.csv")
df_features_ticker = feature_engineering_ticker(df_ticker)
# df_features_ticker['window_start'] = pd.to_datetime(df_features_ticker['window_start'])
df_features_ticker['window_end'] = pd.to_datetime(df_features_ticker['window_end'])
df_features_ticker.to_csv(working_directory + "output\\{}_sum_ticker_10m_{}_v{}.csv".format(coin, data_id, vn), index=False)

df_features_orderbook = feature_engineering_orderbook(df_orderbook)
df_features_orderbook.to_csv(working_directory + "output\\{}_sum_orderbook_10m_{}_v{}.csv".format(coin, data_id, vn), index=False)
# df_features_orderbook = pd.read_csv("output\\BTC_sum_orderbook_10m_202312122122_v7.csv")

# df_features_orderbook = df_features_orderbook.drop(columns = ['window_end', 'time_id'])
df_features_orderbook = df_features_orderbook.drop(columns = ['window_start', 'time_id'])

# df_features_orderbook['window_start'] = pd.to_datetime(df_features_orderbook['window_start'])
df_features_orderbook['window_end'] = pd.to_datetime(df_features_orderbook['window_end'])

combined_result_df = pd.merge(df_features_ticker, df_features_orderbook, on='window_end', suffixes=('_ticker', '_orderbook'))

## 추가변수 만들기
# combined_result_df['realized_volatility_30s'] = np.nan
# combined_result_df['dv4_realized_volatility_30s'] = np.nan
combined_result_df['prices_30s_for_NN_onlyprices'] = np.nan
combined_result_df['prices_30s_for_NN_onlyprices'] = combined_result_df['prices_30s_for_NN_onlyprices'].astype(str)

num_rows = combined_result_df.shape[0]
for i in range(num_rows):
    temp_series = combined_result_df['prices_30s_for_NN'].iloc[i]
    # # Split the string into a list based on the "\n" marker
    # tokenized_list = temp_series.split("\n")
    # # Remove any leading or trailing whitespace from each element
    # tokenized_list = [item.strip() for item in tokenized_list]
    # # Assuming you already have a list named "numbers"
    # if tokenized_list:
    #     tokenized_list.pop(0)  # This will remove the first item in the list
    #     tokenized_list.pop(-1)  # This will remove the first item in the list
    # # Extract the return numbers using list comprehension
    # return_numbers = [float(item.split()[-1]) for item in tokenized_list]
    # return_numbers = temp_series
    # temp_len = len(return_numbers)
    # squared_numbers = [x ** 2 for x in return_numbers]
    # sum_of_squared_numbers = sum(squared_numbers)
    # mean_of_sum_of_squared_numbers = sum_of_squared_numbers/temp_len
    # temp_vol = mean_of_sum_of_squared_numbers ** 0.5
    # combined_result_df.at[i, 'realized_volatility_30s'] = temp_vol
    return_numbers_str = ' '.join(str(num) for num in temp_series)
    combined_result_df.at[i, 'prices_30s_for_NN_onlyprices'] = return_numbers_str
    gc.collect()

# combined_result_df['dv4_realized_volatility_30s'] = combined_result_df['realized_volatility_30s'].shift(-10)

## Additional feature engineering (ex: TVLQ)
combined_result_df['trade.tau'] = np.sqrt(1 / combined_result_df['num_trades'])

combined_result_df['tvpl1'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_1']
combined_result_df['tvpl2'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_2'] 
combined_result_df['tvpl5'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_5'] 
# combined_result_df['tvpl10'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_10'] 
# combined_result_df['tvpl15'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_15']

combined_result_df['tvpl_ep1'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_1']
combined_result_df['tvpl_ep2'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_2']
combined_result_df['tvpl_ep5'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_5']
# combined_result_df['tvpl_ep10'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_10']
# combined_result_df['tvpl_ep15'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_15']

# 아래 작업시 빠른 작업을 위해 10분씩 정리된 dataframe을 중간 저장.
combined_result_df.to_csv(working_directory + "output\\{}_sum_both_10m_{}_v{}.csv".format(coin, data_id, vn), index=False)

c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


###interval: 60


c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
###interval: 300
df_orderbook_pd2.shape: (10323127, 100)
feature expansion from orderbook #1
main iteration: 0


c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in float_scalars


main iteration: 1000
main iteration: 2000
main iteration: 3000
main iteration: 4000
main iteration: 5000
main iteration: 6000
main iteration: 7000
main iteration: 8000
main iteration: 9000
main iteration: 10000
main iteration: 11000
main iteration: 12000
main iteration: 13000
main iteration: 14000
main iteration: 15000
main iteration: 16000
main iteration: 17000
main iteration: 18000
main iteration: 19000
main iteration: 20000
main iteration: 21000
main iteration: 22000
main iteration: 23000
main iteration: 24000
main iteration: 25000
main iteration: 26000
main iteration: 27000
main iteration: 28000
main iteration: 29000
main iteration: 30000
main iteration: 31000
main iteration: 32000
main iteration: 33000
main iteration: 34000
main iteration: 35000
main iteration: 36000
main iteration: 37000
main iteration: 38000
main iteration: 39000
main iteration: 40000
main iteration: 41000
main iteration: 42000
main iteration: 43000
main iteration: 44000
main iteration: 45000
main iteration: 460

c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:291: RuntimeWarning: divide by zero encountered in float_scalars


subperiod_60 iteration: 1000
subperiod_60 iteration: 2000
subperiod_60 iteration: 3000
subperiod_60 iteration: 4000
subperiod_60 iteration: 5000
subperiod_60 iteration: 6000
subperiod_60 iteration: 7000
subperiod_60 iteration: 8000
subperiod_60 iteration: 9000
subperiod_60 iteration: 10000
subperiod_60 iteration: 11000
subperiod_60 iteration: 12000
subperiod_60 iteration: 13000
subperiod_60 iteration: 14000
subperiod_60 iteration: 15000
subperiod_60 iteration: 16000
subperiod_60 iteration: 17000
subperiod_60 iteration: 18000
subperiod_60 iteration: 19000
subperiod_60 iteration: 20000
subperiod_60 iteration: 21000
subperiod_60 iteration: 22000
subperiod_60 iteration: 23000
subperiod_60 iteration: 24000
subperiod_60 iteration: 25000
subperiod_60 iteration: 26000
subperiod_60 iteration: 27000
subperiod_60 iteration: 28000
subperiod_60 iteration: 29000
subperiod_60 iteration: 30000
subperiod_60 iteration: 31000
subperiod_60 iteration: 32000
subperiod_60 iteration: 33000
subperiod_60 iterat